In [1]:
import os
from tempfile import tempdir
from time import time
from turtle import color
from unittest.mock import NonCallableMagicMock
import pandas as pd
import numpy as np
import warnings
import math
import re

import seaborn as sns
warnings.filterwarnings("ignore")

import sys
sys.path.append("./")

from pathlib import Path

import tqdm
import importlib

import matplotlib.pyplot as plt
# from PySide2.QtWidgets import QFileDialog, QApplication
# from PySide2.QtCore import Qt
from copy import copy 

from corems.mass_spectra.input import rawFileReader
from corems.molecular_id.factory.classification import HeteroatomsClassification, Labels
from corems.molecular_id.search.priorityAssignment import OxygenPriorityAssignment
from corems.molecular_id.search.molecularFormulaSearch import SearchMolecularFormulas
from corems.encapsulation.factory.parameters import MSParameters
from corems.encapsulation.constant import Atoms
from corems.mass_spectrum.calc.Calibration import MzDomainCalibration

import corems.lc_icpms_ftms.calc.lc_icrms_qc_assign as icrms
import corems.lc_icpms_ftms.calc.lc_icrms_helpers as lcmsfns

import importlib


In [2]:
# get files & parsers

dir='/Volumes/Samsung_T5/NHMFL/2023_January_Christian/wastewater/Zorbax_5um_inject_1500ms/neg'
flist=os.listdir(dir)
os.chdir(dir)

# get parsers

MSfiles={}

for f in flist:
    if ('.raw' in f) & ('._' not in f) & ('qh2o' not in f):
        print(f)
        parser = rawFileReader.ImportMassSpectraThermoMSFileReader(f)
        MSfiles[f]={'parser': parser}

data = icrms.lc_icr_assign(dir)

data.master_data_holder = MSfiles

data._raw_filelist = data.master_data_holder.keys()

20230113_LBA_Boiteau_Zorbax5um_1500IT_neg_FinalEff_300_500_02.raw
20230113_LBA_Boiteau_Zorbax5um_1500IT_neg_FinalEff_300_500_01.raw
20230113_LBA_Boiteau_Zorbax5um_1500IT_neg_FinalEff_fullmz_02.raw
20230113_LBA_Boiteau_Zorbax5um_1500IT_neg_NovWastewater_300_500_02.raw
20230113_LBA_Boiteau_Zorbax5um_1500IT_neg_NovWastewater_300_500_01.raw
20230113_LBA_Boiteau_Zorbax5um_1500IT_neg_NovWastewater_fullmz_02.raw
20230113_LBA_Boiteau_Zorbax5um_1500IT_neg_NovWastewater_fullmz_01.raw
20230113_LBA_Boiteau_Zorbax5um_1500IT_neg_FinalEff_fullmz_01.raw


In [3]:
# set assignment parameters
MSParameters.mass_spectrum.threshold_method = 'signal_noise'
MSParameters.mass_spectrum.s2n_threshold = 2
MSParameters.ms_peak.peak_min_prominence_percent = 0.001

MSParameters.molecular_search.error_method = 'None'
MSParameters.molecular_search.min_ppm_error = -0.25
MSParameters.molecular_search.max_ppm_error = 0.25

MSParameters.molecular_search.isProtonated = True
MSParameters.molecular_search.isRadical = False
MSParameters.molecular_search.isAdduct = False

MSParameters.molecular_search.score_method = "prob_score"
MSParameters.molecular_search.output_score_method = "prob_score"

MSParameters.molecular_search.url_database = None
MSParameters.molecular_search.min_dbe = -1
MSParameters.molecular_search.max_dbe = 20

MSParameters.molecular_search.usedAtoms['C'] = (1,50)
MSParameters.molecular_search.usedAtoms['H'] = (4,100)
MSParameters.molecular_search.usedAtoms['O'] = (1,20)
MSParameters.molecular_search.usedAtoms['N'] = (0,3)
#MSParameters.molecular_search.usedAtoms['I'] = (0,0)
#MSParameters.molecular_search.usedAtoms['F'] = (0,0)
#MSParameters.molecular_search.usedAtoms['S'] = (0,0)
#MSParameters.molecular_search.usedAtoms['P'] = (0,1)


In [4]:
# run assignment algorithm
timerange = [8,22]  # start (min) to stop (min)
interval = 2  # min 
refmasslist = "/Users/christiandewey/CoreMS/db/Hawkes_neg.ref"

data.assign_formula(interval = interval, timerange = timerange, refmasslist = refmasslist, calorder =2, charge = -1)
#data_df = lcmsfns.assign_formula(parser=parser, interval = interval, timerange = timerange, refmasslist = refmasslist, calorder =2, charge = -1)

data_df = data.complete_results_df



20230113_LBA_Boiteau_Zorbax5um_1500IT_neg_FinalEff_300_500_02.raw
1 of 8 files
timestart: 8
negative
MS Obj loaded - 461 peaks found.
MS Obj loaded - 461 peaks found.
Reference mass list loaded - 2030 calibration masses loaded.
1 calibration points matched within thresholds.


100%|██████████| 80/80 [00:00<00:00, 557.63it/s]

Started database insert using 2075 iterations for a total of 103735 rows



2075it [00:18, 114.63it/s]


'runworker'  18590.19 ms


...

'run_molecular_formula'  2594.40 ms
641 assigned peaks and 344 unassigned peaks, total  = 65.08 %, relative abundance = 74.82 %, RMS error (best candidate) (ppm) = 0.058
timestart: 18
negative
MS Obj loaded - 873 peaks found.
MS Obj loaded - 873 peaks found.
Reference mass list loaded - 2030 calibration masses loaded.
160 calibration points matched within thresholds.
minimize function completed with RMS error of: 0.049 ppm
minimize function performed 3000 fn evals and 65 iterations
'runworker'  10.47 ms


Loading molecular formula database: 66938it [00:02, 31408.46it/s]
Started molecular formula search for class {"N": 3, "O": 20}, (de)protonated : 100%|██████████| 80/80 [00:00<00:00, 204.48it/s]


'run_molecular_formula'  2570.53 ms
602 assigned peaks and 271 unassigned peaks, total  = 68.96 %, relative abundance = 75.71 %, RMS error (best candidate) (ppm) = 0.057
timestart: 20
negative
MS Obj loaded - 1023 peaks found.
MS Obj loaded - 1023 peaks found.
Reference mass list loaded - 2030 calibration masses loaded.
163 calibration points matched within thresholds.
minimize function completed with RMS error of: 0.047 ppm
minimize function performed 3005 fn evals and 64 iterations
'runworker'  3.70 ms


Loading molecular formula database: 66938it [00:02, 30000.88it/s]
Started molecular formula search for class {"N": 3, "O": 20}, (de)protonated : 100%|██████████| 80/80 [00:00<00:00, 177.88it/s]


'run_molecular_formula'  2724.80 ms
685 assigned peaks and 338 unassigned peaks, total  = 66.96 %, relative abundance = 69.07 %, RMS error (best candidate) (ppm) = 0.059
